#### - Sobhan Moradian Daghigh
#### - 12/29/2021
#### - PR - EX02 - Q5 - All Parts

In [1]:
import numpy as np
from PIL import Image
import os, io
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import multivariate_normal
from sklearn.metrics import accuracy_score

In [2]:
dataset = pd.read_csv (r'./inputs/P5/blood_test.txt')
dataset

,mcv,alkphos,sgpt,sgot,gammagt,drinks,selector
0,85,92,45,27,31,0.0,1
1,85,64,59,32,23,0.0,2
2,86,54,33,16,54,0.0,2
3,91,78,34,24,36,0.0,2
4,87,70,12,28,10,0.0,2
...,...,...,...,...,...,...,...
340,99,75,26,24,41,12.0,1
341,96,69,53,43,203,12.0,2
342,98,77,55,35,89,15.0,1
343,91,68,27,26,14,16.0,1


In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 345 entries, 0 to 344
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   mcv       345 non-null    int64  
 1   alkphos   345 non-null    int64  
 2   sgpt      345 non-null    int64  
 3   sgot      345 non-null    int64  
 4   gammagt   345 non-null    int64  
 5   drinks    345 non-null    float64
 6   selector  345 non-null    int64  
dtypes: float64(1), int64(6)
memory usage: 19.0 KB


In [4]:
dataset.loc[dataset.drinks > 5 , 'addicted'] = 1
dataset.loc[dataset.drinks <= 5, 'addicted'] = 0

dataset

,mcv,alkphos,sgpt,sgot,gammagt,drinks,selector,addicted
0,85,92,45,27,31,0.0,1,0.0
1,85,64,59,32,23,0.0,2,0.0
2,86,54,33,16,54,0.0,2,0.0
3,91,78,34,24,36,0.0,2,0.0
4,87,70,12,28,10,0.0,2,0.0
...,...,...,...,...,...,...,...,...
340,99,75,26,24,41,12.0,1,1.0
341,96,69,53,43,203,12.0,2,1.0
342,98,77,55,35,89,15.0,1,1.0
343,91,68,27,26,14,16.0,1,1.0


In [5]:
groups = dataset.groupby(by='selector')
groups.first()

,mcv,alkphos,sgpt,sgot,gammagt,drinks,addicted
selector,,,,,,,
1,85,92,45,27,31,0.0,0.0
2,85,64,59,32,23,0.0,0.0


In [6]:
sets = []
for grp, pdf in groups:
    df = pd.DataFrame(pdf)
    sets.append(df)

train = sets[0]
test = sets[1]

In [7]:
train.head()

,mcv,alkphos,sgpt,sgot,gammagt,drinks,selector,addicted
0,85,92,45,27,31,0.0,1,0.0
6,88,62,20,17,9,0.5,1,0.0
7,88,67,21,11,11,0.5,1,0.0
8,92,54,22,20,7,0.5,1,0.0
9,90,60,25,19,5,0.5,1,0.0


In [8]:
samples10 = train.sample(10, random_state=1)
samples50 = train.sample(50, random_state=2)
samples100 = train.sample(100, random_state=3)

### Part A.

In [9]:
def get_meanCov(datas):
    if datas.empty:
        return [0] * 6, [[0] * 6] * 6
    datas = datas.iloc[:, :-2]
    mean = np.array(datas.mean())
    cov = np.divide(np.dot(np.transpose(np.subtract(datas, mean)), np.subtract(datas, mean)), datas.shape[0])
    return mean, cov

In [10]:
def MLH_params(dataset):
    mcs = []
    mcs.append(get_meanCov(dataset.loc[dataset.addicted == 1]))
    mcs.append(get_meanCov(dataset.loc[dataset.addicted == 0]))
    return mcs

In [11]:
for frame in [dataset, samples10, samples50, samples100]:
    temp = MLH_params(frame)
    print("- [{} Samples]".format(len(frame)))
    for i, cls in zip(range(2), ['Addicted', 'NoAddicted']):
        print("- Class: {}".format(cls))
        print("Mean:")
        df = pd.DataFrame(temp[i][0]).T
        display(df.round(3))
        print("\nCov:")
        df = pd.DataFrame(temp[i][1])
        display(df.round(3))
#         pprint(temp[i][1])
        print('\n-----------------------------------------')

- [345 Samples]
- Class: Addicted
Mean:


,0,1,2,3,4,5
0,92.523,72.886,36.727,28.886,57.17,7.989



Cov:


,0,1,2,3,4,5
0,16.590,9.605,9.404,11.514,64.661,1.267
1,9.605,391.692,41.105,40.885,179.281,14.090
2,9.404,41.105,436.653,151.219,504.535,13.338
3,11.514,40.885,151.219,123.283,318.406,6.828
4,64.661,179.281,504.535,318.406,2456.073,37.945
5,1.267,14.090,13.338,6.828,37.945,8.375



-----------------------------------------
- Class: NoAddicted
Mean:


...


-----------------------------------------
- Class: NoAddicted
Mean:


,0,1,2,3,4,5
0,89.397,72.912,26.603,20.441,22.294,1.441



Cov:


,0,1,2,3,4,5
0,12.710,8.976,2.908,2.207,1.442,0.538
1,8.976,320.963,-9.564,21.304,76.482,-4.255
2,2.908,-9.564,84.622,15.087,93.367,0.749
3,2.207,21.304,15.087,32.599,24.738,0.055
4,1.442,76.482,93.367,24.738,428.561,-0.792
5,0.538,-4.255,0.749,0.055,-0.792,1.607



-----------------------------------------


### Part B.

In [32]:
def cal_prior_probabilities(dataset):
    return np.log(dataset.groupby(by = 'addicted').apply(lambda x: np.divide(len(x), dataset.shape[0]))).values

In [33]:
# Continues with Noraml Distribution
def cal_probability_density(mean, cov, x):
    var = multivariate_normal(mean=mean, cov=cov, allow_singular=True)
    prob = var.pdf(x)
    return prob

In [34]:
def cal_likelihood_probabilities(test_row, mean, cov):
    
    likelihood_probabilities = []
    
    for i in range(2): # Unique labels
        likelihood = 0
        likelihood = np.log(cal_probability_density(mean[i], cov[i], test_row))
        likelihood_probabilities.append(likelihood)

    return likelihood_probabilities

In [35]:
def BC(test_dataset, periors, mean, cov):
    predictions = []
    for i in range(test_dataset.shape[0]):
        
        likelihood = cal_likelihood_probabilities(test_dataset.iloc[i, :], mean, cov)

        # log(a*b) = loga + logb
        probabilities = periors + likelihood
        
        mx_idx = np.argmax(probabilities)
        predictions.append([0, 1][mx_idx])

    return predictions

In [36]:
periors = cal_prior_probabilities(train)

for frame in [samples10, samples50, samples100]:
    temp = MLH_params(frame)
    mean, cov = [temp[0][0], temp[1][0]], [temp[0][1], temp[1][1]]
    predictions = BC(test.iloc[:, :-2], periors, mean, cov)
    accuracy = accuracy_score(test.iloc[:, -1], predictions)
    print('accuracy: {:.3f}'.format(accuracy))

accuracy: 0.785
accuracy: 0.135
accuracy: 0.230
